In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
import pickle

import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
high_model = pickle.load(open('high_model', 'rb'))
low_model = pickle.load(open('low_model', 'rb'))
pressure_model = pickle.load(open('pressure_model', 'rb'))

In [ ]:
import datetime
import requests
import math

def get_ip():
    api_url = "https://ipinfo.io/json"
    print("requesting ip")
    response = requests.get(api_url)
    print("done requesting ip")
    ip = response.json()["ip"].split(",")
    return str(ip[0])

def get_wet(temp, hum):
    return temp * math.atan(0.152*(hum+8.3136)**2)+math.atan(temp+hum)-math.atan(hum-1.6763)+0.00391838*hum**(3/2)*math.atan(.0231*hum)-4.686


ip = get_ip()

now = datetime.datetime.now()
days = []
days.append(str(now.replace(hour=0, minute=0, second=0, microsecond=0) - datetime.timedelta(days=1))[:11])
days.append(str(now.replace(hour=0, minute=0, second=0, microsecond=0) - datetime.timedelta(days=2))[:11])
days.append(str(now.replace(hour=0, minute=0, second=0, microsecond=0) - datetime.timedelta(days=3))[:11])
days.append(str(now.replace(hour=0, minute=0, second=0, microsecond=0) - datetime.timedelta(days=4))[:11])
print(ip)
built_api_url = "http://api.weatherapi.com/v1/history.json?key=fa024d2a350b4df484c42502232603 &q=" + ip + "&dt="
temp_mins =[]
temp_maxs = []
winds = []
humidities=[]
wet_lows=[]
wet_highs=[]
count = 0
pressures=[]
for day in days:
    response = requests.get(built_api_url+day)
    j = response.json()
    min = j["forecast"]["forecastday"][0]["day"]["mintemp_f"]
    temp_mins.append(min)
    max = j["forecast"]["forecastday"][0]["day"]["maxtemp_f"]
    temp_maxs.append(max)
    wind = j["forecast"]["forecastday"][0]["day"]["maxwind_mph"]
    winds.append(wind)
    humidity = j["forecast"]["forecastday"][0]["day"]["avghumidity"]
    humidities.append(humidity)
    pressure = j["forecast"]["forecastday"][0]["hour"][0]["pressure_in"]
    pressures.append(pressure)
    wet_highs.append(get_wet(max, humidity))
    wet_lows.append(get_wet(min, humidity))
    count += 1

In [ ]:
past_days = pd.DataFrame([[temp_maxs[0], temp_maxs[1], temp_maxs[2], temp_maxs[3], temp_mins[0], temp_mins[1], temp_mins[2], temp_mins[3], wet_highs[0], wet_highs[1], wet_highs[2], wet_highs[3], wet_lows[0], wet_lows[1], wet_lows[2], wet_lows[3], humidities[0], humidities[1], humidities[2], humidities[3], winds[0], winds[1], winds[2], winds[3], pressures[0], pressures[1], pressures[2], pressures[3]]], columns=['High-1', 'High-2', 'High-3', 'High-4',
                'Low-1', 'Low-2', 'Low-3', 'Low-4',
                'WetBulb_High-1', 'WetBulb_High-2', 'WetBulb_High-3', 'WetBulb_High-4',
                'WetBulb_Low-1', 'WetBulb_Low-2', 'WetBulb_Low-3', 'WetBulb_Low-4',
                'Humidity-1', 'Humidity-2', 'Humidity-3', 'Humidity-4',
                'WindSpeed-1', 'WindSpeed-2', 'WindSpeed-3', 'WindSpeed-4',
                'Pressure-1', 'Pressure-2', 'Pressure-3', 'Pressure-4'])

In [ ]:
high_prediction = high_model.predict(past_days)[0]
w1 = widgets.FloatText(value=high_prediction)
low_prediction = low_model.predict(past_days)[0]
w2 = widgets.FloatText(value=low_prediction)
pressure_prediction = pressure_model.predict(past_days)[0]
w3 = widgets.FloatText(value=pressure_prediction)

In [ ]:
text_1 = widgets.HTML(value="<h1>The high for today is: </h1>")
text_2 = widgets.HTML(value="<h2>The low for today is: </h2>")
text_3 = widgets.HTML(value="<h3>The low for today is: </h3>")

vbox_text = widgets.VBox([text_1, w1, text_2, w2, text_3, w3])

In [ ]:
page = widgets.HBox([vbox_text])
display(page)